## 第52条 用subprocess管理子进程

由python启动的子进程可以平行地运行，这让我们能够充分利用计算机的每一个CPU核心，来尽量提升程序的处理效率。

In [1]:
import subprocess

result = subprocess.run(
    ['echo', 'Hello from the child'], # ehco 通常用于在屏幕上显示文本
    capture_output=True,
    encoding='utf-8'
)

result.check_returncode() # No exception means clean exit
print(result.stdout)

Hello from the child



子进程可以独立于父进程而运行，假如刚才那条子进程不是通过run函数启动，而是由Popen类启动，那么我们就可以在它启动之后，让Python程序去做别的任务。

In [10]:
proc = subprocess.Popen(['sleep', '0.001']) # subprocess.Popen()用于启动一个新的进程来执行指定的命令。
                                            # 在这里，它启动了一个名为sleep的命令，'0.001'是sleep命令的参数，表示程序将会休眠0.001秒

while proc.poll() is None: # proc.poll()返回进程的退出代码。如果进程尚未结束，它会返回None
    print('Working')

    # Some time-consuming work here

print('Exit status', proc.poll())

Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Working
Exit status 0


把子进程从父进程中剥离，可以让程序平行地运行多条子进程。

In [12]:
import time

start = time.time()
sleep_procs = []
for _ in range(10):
    proc = subprocess.Popen(['sleep', '1'])
    sleep_procs.append(proc)

在主进程里调用每条子进程的communicate方法，等待这条子进程把它的I/O工作处理完毕。

In [13]:
for proc in sleep_procs:
    proc.communicate()

end = time.time()
delta = end - start
print(f'Finished in {delta: 3} seconds')

Finished in  1.048064947128296 seconds


我们还可以在python程序中把数据通过管道发送给子进程所运行的外部命令，然后将那条命令的输出结果获取到python程序中。而且，在执行外部命令这个环节中，可以并行地运行多条命令。

In [14]:
import os

def run_encrypt(data):

    env = os.environ.copy() # 这里创建了一个环境变量的副本，env将被传递给后续的subprocess.Popen调用，以确保子进程能够获取到相同的环境变量

    env['password'] = 'zf7ShyBhZOraQDsE/FiZpm/m/8f9X+M1'
    proc = subprocess.Popen(
        ['openssl', 'enc', '-des3', '-pass', 'env:password'],
        env=env,
        stdin=subprocess.PIPE, # 这将配置子进程的输入和输出流
        stdout=subprocess.PIPE
    )

    proc.stdin.write(data)
    proc.stdin.flush() # Ensure that the child gets input

    return proc # 返回了表示子进程的Popen对象，使得调用者可以与子进程进行交互

随机生成一些字节数据，并传给刚才的加密函数去处理。这些子进程能够并行的运行，并各自处理他所接到的那份数据。

In [15]:
procs = []
for _ in range(3):
    data = os.urandom(10)
    proc = run_encrypt(data)
    procs.append(proc)

for proc in procs:
    out, _ = proc.communicate()
    print(out)

b'Salted__\x99\xee\x15a\xd8\xdc@\xeb\xbb\xbc]\xbb\xbf\xb4#]\x19\xab\xb6\xae\x91\x92\xa3\xb7'
b'Salted__\xff{\x02Be\\\xc5\x80\xdb\x96\xb3\xec\x0e\x9f\x8aF6\xd0\xf9Y{\xaf\x0f('
b'Salted__\xd1p\xaf\xcb/\x83;Y\x18\x85XL\x86\xed\x08\xce\x9cv\x076\xf3\xd19)'


这些并行的子进程还可以分别与另一套平行的子进程对接，形成许多条平行的管道（pipe）。这种管道与UNIX管道类似，能够把一条子进程的输出端与另一条子进程的输入端连接起来。

In [16]:
def run_hash(input_stdin):
    return subprocess.Popen(
        ['openssl', 'dgst', '-whirlpool', '-binary'],
        stdin=input_stdin,
        stdout=subprocess.PIPE
    )

先启动一批进程来加密数据，然后启动另一批进程根据前面那些进程的加密结果形成哈希码。请注意，前面那批进程（也就是上游进程）的stdout实例必须谨慎处理，它把相应的哈希进程（也就是下游进程）启动之后，就应该及时关闭（close），并将它设为None。

In [23]:
encrypt_procs = []
hash_procs = []

for _ in range(3):
    data = os.urandom(100)

    encrypt_proc = run_encrypt(data)
    encrypt_procs.append(encrypt_proc)

    hash_proc = run_hash(encrypt_proc.stdout)
    hash_procs.append(hash_proc)

    encrypt_proc.stdout.close()
    encrypt_proc.stdout = None

只要上游、下游的子进程都启动起来，两者之间的I/O管道就会自动打通。

In [24]:
for proc in encrypt_procs:
    proc.communicate()
    assert proc.returncode == 0

for proc in hash_procs:
    out, _ = proc.communicate()
    print(out[-10:])
    assert proc.returncode == 0

b'o\x8c\xab\xd2\xad\xbd\x91\x04\xee\xd1'
b'\x9a$r\x84\xdej7h\x9f;'
b'N\x9d\xf1\x00ISsa\xa3\xc7'


子进程有可能一直不结束，或者由于某种原因卡在输入端或者输出端，那么可以在调用communicate方法时指定timeout参数。

In [25]:
proc = subprocess.Popen(['sleep', '10'])

try:
    proc.communicate(timeout=0.1)
except subprocess.TimeoutExpired:
    proc.terminate()
    proc.wait()

print('Exit status', proc.poll())

Exit status -15
